In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Goto https://keras.io/api/applications/ for getting any transfer learning model based on ImageNet
"""ImageNet is a large-scale image database that was created to serve as a benchmark for visual object recognition research.
It contains over 14 million labeled images spanning 22,000 categories.The ImageNet project gained significant attention in the
computer vision community when it launched the Large Scale Visual Recognition Challenge (ILSVRC) in 2010, which was a
competition designed to evaluate the accuracy of algorithms for object detection and image classification tasks. One of the
key outcomes of the ILSVRC competition was the development of deep convolutional neural networks (CNNs), such as AlexNet, VGG,
and ResNet, which dramatically improved the state-of-the-art performance on object recognition tasks."""

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
#from keras.applications.resnet import ResNet50
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/train'
valid_path = '/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/test'

# add preprocessing layer to the front of VGG(include=False means donot add the last layer)
vgg = VGG16(input_shape=[224,224,3], weights='imagenet', include_top=False)



ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (/usr/local/lib/python3.10/dist-packages/keras/api/preprocessing/image/__init__.py)

In [ ]:
# execute this only once... if you run again, skip this part
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/Artificial_Intelligence_Engineering/CV/5_train.zip", 'r') as zip_ref:        # get the path for train.zip using copypath. paste the link within the quotation
     # creates new folder 'cats_dogs_images' and saves all images present in train.zip
    zip_ref.extractall("/content/extracted_images")  # create a new folder, rename it and get the path of it using copypath.. paste the link here



In [ ]:
import os, shutil, pathlib

original_dir = pathlib.Path("/content/extracted_images/train") # get the path of train folder in cats_dogs_images using copypath and paste the link here
new_base_dir = pathlib.Path(r"/content/extracted_images/dataset") # create a new folder, rename it and get the path using copypath and paste the link here.

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

# calling the function thrice to create 3 subsets.
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [ ]:
import os
import cv2
labels = ['cat', 'dog']
img_size = 224
def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data,dtype='object')

In [ ]:
train = get_data(r'/content/extracted_images/dataset/train')
test = get_data(r'/content/extracted_images/dataset/test')

In [ ]:
print(len(train))
print(len(test))

2000
2000


In [ ]:
x_train = []
y_train = []
x_val = []
y_val = []

for feature, label in train:
  x_train.append(feature)
  y_train.append(label)

for feature, label in test:
  x_val.append(feature)
  y_val.append(label)

# Normalize the data
x_train = np.array(x_train).astype('float32') / 255
x_val = np.array(x_val).astype('float32') / 255

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
len(train[0])

In [ ]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/train')

In [ ]:
type(folders)

list

['/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/train']

In [ ]:
folders = glob('/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/train')


# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
glob('/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/train/*')

['/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/train/cat',
 '/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/train/dog']

In [ ]:
folders = glob('/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/train/*')
print(type(folders))
print(folders[0])
print(len(folders))

In [ ]:

# useful for getting number of classes
folders = glob('/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/train/*')





from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/CNN/cats_dogs_images/image_seggrated/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')



Found 1073 images belonging to 2 classes.
Found 1133 images belonging to 2 classes.


In [ ]:
1073/32

33.53125

In [ ]:
len(training_set)

34

In [ ]:
type(training_set)

In [ ]:
len(training_set)

In [ ]:
# fit the model
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
34/34 [==============================] - 1347s 40s/step - loss: 0.8036 - accuracy: 0.5000 - val_loss: 0.7231 - val_accuracy: 0.5000
Epoch 2/5
34/34 [==============================] - 1271s 38s/step - loss: 0.7081 - accuracy: 0.5000 - val_loss: 0.7061 - val_accuracy: 0.5000
Epoch 3/5
34/34 [==============================] - ETA: 0s - loss: 0.7012 - accuracy: 0.5000 

In [ ]:
'''r=model.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 5,
                         validation_data = test_set,
                         nb_val_samples = 2000)'''


# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

import tensorflow as tf

from keras.models import load_model

model.save('facefeatures_new_model.h5')